# Sparse dowker homology

In this tutorial we describe the use of the python package [dowker_homology](https://github.com/mbr085/Sparse-Dowker-Nerves) to calculate the persistent homology of point clouds. Running the entire workbook will take approximately ten minutes on a modern laptop.

## Čech persistent homology
We start by calculating Čech persistent homology of a point cloud in ℝ<sup>4</sup>. First we import the relevant python packages, set a random seed and generate the sample data used for the examples below. The first data we use are 100 points on the Clifford torus. 

In [ ]:
import time
start = time.time()

In [ ]:
%%time
# import packages
import numpy as np
import urllib.request
from io import BytesIO
import pandas as pd
from scipy.spatial.distance import cdist
import dowker_homology as dh
np.random.seed(1)

In [ ]:
# generate data
def clifford_torus(N):
    x = np.linspace(0, 2*np.pi, num=N, endpoint=False).reshape(N,1)
    y = np.sqrt(N)*x
    return np.hstack((np.cos(x), np.sin(x), np.cos(y), np.sin(y)))

coords = clifford_torus(100)

In order to calculate the persistent homology of a point cloud with an ambient Čech complex, we create a `Dowker_Ambient` object. By default the `plot_persistence` method takes as input an Euclidean point cloud X and returns the persistence diagram of the associated ambient Čech complex.

In [ ]:
%%time
# initiate dowker homology object
dowker = dh.Dowker_Ambient()
# calculate and plot persistent homology
dowker.plot_persistence(X=coords)

Information about the sizes of the nerves can be obtained by the `cardinality_information` method.

In [ ]:
# look at nerve sizes
dowker.cardinality_information()

Note that we have plotted the persistence classes with transparent big dots. This allows us to see that there are two persistence classes with birth approximately 0.046 and death approximately 1. It is possible to zoom in and to make the dots more transparent: 

In [ ]:
# plot persistent homology
dowker.plot_persistence(zoom=True, alpha=0.2)

Note that we did not specify the data X the second time we plotted the persistence diagram. This lets the software reuse results of the previous calculations. If X is specified again, all calculations are repeated. 

In addition to the `Dowker_Ambient` constructor, we provide `Dowker` and `Dowker_Intrinsic` constructors for calculating persistent homology of nerves of dowker dissimilarities and intrinsic nerves of metric spaces. 

The following are the parameters that can be passed to all constructors together with their default values. We will walk through all of these in this tutorial.

```python
dimension=1
translation_function=None
initial_point=0
resolution=0
n_samples=None
multiplicative_interleaving=1
additive_interleaving=0
homology_method='dual'
max_simplex_size=2e5
```

In addition the `Dowker_Intrinsic` has a dissimilarity argument, which defaults to 'euclidean'. 
```python
dissimilarity='euclidean'
```

To begin with, `homology_method` can be either 'standard' or 'dual', where 'dual' means computing cohomology. See [PHAT](https://bitbucket.org/phat-code/phat) for more information. 

The `max_simplex_size` parameter helps by giving an error if the resulting nerve is very large and calculations will take a long time. Then the options are to either set `max_simplex_size` higher and expect some waiting time or to change the interleaving parameters so that calculations finish after reasonably short times. 

Two important optional parameters for Dowker objects are the maximal homology dimension `dimension` and the number `n_samples` of vertices in the nerve. We use a farthest point sampling to reduce the number of vertices. 

In [ ]:
%%time
# initiate dowker homology object
dowker = dh.Dowker_Ambient(dimension=2, n_samples=40)
# calculate and plot persistent homology
dowker.plot_persistence(X=coords)

After plotting the persistence diagram the object dowker contains 
a fair amount of variables. 
Most prominently the `homology` variable is a numpy array with three columns.<br>
The first column gives homological dimension.<br>
The second column gives birth.<br>
The third column gives death.

In [ ]:
# first rows of persistence diagram
pd.DataFrame(dowker.homology, 
             columns=['dim', 'birth', 'death']).head()

If we like we can add a persistence column.

In [ ]:
# add persistence column
homology = pd.DataFrame(dowker.homology, 
                        columns=['dim', 'birth', 'death'])
homology['pers'] = homology['death'] - homology['birth']
homology.head()

## Intrinsic Čech complex

Instead of the ambient Čech complex
we can compute the intrinsic Čech complex.
The ambient Čech complex is the nerve of a Dowker dissimilarity of the form  X × ℝ<sup>d</sup> → [0, ∞] and the intrinsic Čech complex is the nerve of a Dowker dissimilarity of the form X × X → [0, ∞]. For the intrinsic Čech complex we can specify all metrics accepted by `scipy.spatial.distance.cdist` in addition to `'precomputed'`. 

In [ ]:
%%time
# initiate dowker homology object
dowker = dh.Dowker_Intrinsic(dimension=1,
                             dissimilarity='euclidean')
# calculate and plot persistent homology
dowker.plot_persistence(X=coords)
# show cardinality
dowker.cardinality_information()

Below we compute the Čech homology of the point-cloud given by the vertices of a regular simplex of dimension 999.

In [ ]:
%%time
# initiate dowker homology object
dowker = dh.Dowker_Intrinsic()
# persistence diagram of a simplex
dowker.plot_persistence(X=np.diag(np.ones(1000)))
# cardinality information
dowker.cardinality_information()

## Sparse Dowker Nerves

The sparse version of persistent homology from Example 5.6 in [Sparse Filtered Nerves](https://arxiv.org/abs/1810.02149) is interleaved with Čech persistent homology. The easiest way to use it is to specify the additive and multiplicative interleaving constants. 

In [ ]:
%%time
# initiate dowker homology object
dowker = dh.Dowker_Ambient(dimension=1, 
                           multiplicative_interleaving=1.5, 
                           additive_interleaving=0.1)
# calculate and plot persistent homology
dowker.plot_persistence(X=coords)
# cardinality information
dowker.cardinality_information()

Note that there is a second black line in addition to the diagonal. The interleaving guarantees that all points above this interleaving line will be present in the original persistence diagram as well. 

We can also sparsify the intrinsic Čech complex. 

In [ ]:
%%time
# initiate dowker homology object
dowker = dh.Dowker_Intrinsic(dimension=1, 
                             multiplicative_interleaving=1.5, 
                             additive_interleaving=0.2)
# calculate and plot persistent homology
dowker.plot_persistence(X=coords)
# cardinality information
dowker.cardinality_information()

For data of high dimension we are able to reduce the size of the nerve drastically by using a multiplicative interliaving. We illustrate this on a dataset consisting of random data in 16 dimensional euclidean space. 

In [ ]:
# Download data and transform to numpy array
url = ('https://raw.githubusercontent.com/' +
       'n-otter/PH-roadmap/master/data_sets/' +
       'roadmap_datasets_point_cloud/random_' +
       'point_cloud_50_16_.txt')

response = urllib.request.urlopen(url)
data = response.read()      # a `bytes` object
text = data.decode('utf-8')
coords = np.genfromtxt(BytesIO(data), delimiter=" ")

In [ ]:
%%time
# initiate dowker homology object
dowker = dh.Dowker_Ambient(dimension=7,
                           multiplicative_interleaving=2.5)
# calculate and plot persistent homology
dowker.plot_persistence(X=coords)
dowker.cardinality_information()

## Subspace sparsification

In the introduction of [Sparse Filtered Nerves](https://arxiv.org/abs/1810.02149) we mention the additive interleaving of subspaces with the Hausdorff distance. This additive part can be specified using either the `resolution` or the `n_samples` parameters. 
If there is no other interleaving, the `resolution` parameter is the intersect of the interleaving line with the y-axis. The parameter `n_samples` specifies how many points to sample in a farthest point sampling. 
It is convenient, because computing time is low for few points. It can also be used to find an appropriate value for the `resolution` parameter.
For a general alpha-interleaving the total interleaving is alpha(t + resolution).

Here we use the complete data and specify `n_samples` to be 50.

In [ ]:
coords = clifford_torus(100)

In [ ]:
%%time
# initiate dowker homology object
dowker = dh.Dowker_Intrinsic(dimension=1,
                             n_samples=50,
                             multiplicative_interleaving=1.2)
# calculate and plot persistent homology
dowker.plot_persistence(X=coords)
# cardinality information
dowker.cardinality_information()

Note again that there is a second black line in addition to the diagonal. We know that all points above this interleaving line will be present in the original persistence diagram as well.

From the above persistence diagram we read off that 0.62 is an appropriate value for the `resoultion` parameter. We repeat the computation with `resolution` set to 0.62.

In [ ]:
%%time
# initiate dowker homology object
dowker = dh.Dowker_Intrinsic(dimension=1,
                             resolution=0.62,
                             multiplicative_interleaving=1.2)
# calculate and plot persistent homology
dowker.plot_persistence(X=coords)
# cardinality information
dowker.cardinality_information()

Note that the interleaving line intersects the y-axis at 1.2 * 0.62 = 0.744

## General interleavings

We can also specify interleavings with arbitrary translation functions α(t). Note that there are no automated checks of the correctness of these functions and it is the users responsability to make sure that they are indeed translation functions. The following example is the same as the example with multiplicative interleaving of 2.5 and an additive interleaving of 0.5.

In [ ]:
# generate new data
coords = clifford_torus(5000)

In [ ]:
%%time
# specify translation function
def beta_fct(time):
    return 2.5 * time + 0.5
# initiate dowker homology object
dowker = dh.Dowker_Intrinsic(dimension=1,
                             translation_function=beta_fct,
                             dissimilarity='euclidean')
# calculate and plot persistent homology
dowker.plot_persistence(X=coords)

More complex translation functions can also be specified. 

In [ ]:
%%time
# specify translation function
def beta_fct(time):
    return pow(time, 3) + 0.3
# initiate dowker homology object
dowker = dh.Dowker_Intrinsic(dimension=1,
                             translation_function=beta_fct,
                             resolution=0.3,
                             dissimilarity='euclidean')
# calculate and plot persistent homology
dowker.plot_persistence(X=coords)

Note that here the interleaving guarantee is provided by a curve more general than a straight line.

## Arbitrary Dowker dissimilarity

In the previous examples we have used intrinsic or ambient Čech complex. Here we compute the persistent homology for arbitrary Dowker dissimilarities.

Inspired from the [persnet software](https://github.com/fmemoli/PersNet) we compute Dowker homology of cyclic networks.

In [ ]:
%%time
# generate cyclic network
def cyclic_network(n_nodes):
    m = np.zeros((n_nodes, n_nodes))
    for i in range(n_nodes):
        for j in range(n_nodes):
            m[i, j] = (j - i) % n_nodes
    return m
dowker_dissimilarity = cyclic_network(100)
# initiate dowker homology object
dowker = dh.Dowker(dimension=1)
# calculate and plot persistent homology
dowker.plot_persistence(X=dowker_dissimilarity)

Below we compute persistent homology of the dowker nerve of the function X × G → [0, ∞] specifying the distance between a point x in the Clifford torus X and a point g in a grid G containing X.

In [ ]:
%%time
# initiate dowker homology object
dowker = dh.Dowker(dimension=1)
# grid of witness points
grid = np.array([(x1, x2, x3, x4) for 
                 x1 in np.linspace(-1, 1, 10) for 
                 x2 in np.linspace(-1, 1, 10) for
                 x3 in np.linspace(-1, 1, 10) for
                 x4 in np.linspace(-1, 1, 10)])
# calculate and plot persistent homology
dowker.plot_persistence(X=cdist(clifford_torus(100), grid))

We next compute the intrinsic Čech complex directly from the distance matrix. 

In [ ]:
%%time
# initiate dowker homology object
dowker = dh.Dowker(dimension=1)
# calculate and plot persistent homology
dowker.plot_persistence(X=cdist(clifford_torus(100), clifford_torus(100)))

This whole notebook hopefully takes less than 20 minutes to run on a modern laptop. 

In [ ]:
total_time = time.time() - start
total_time / 60